In [1]:
from datetime import datetime, timezone
import configparser
import json
import requests
import ccxt
import time
from pprint import pprint 
import pandas as pd
from urllib.parse import urljoin 


binance_symbol = 'BTC/USDT'
coinbase_symbol = 'BTC/USD'

coinbase_datetime_format = '%Y-%m-%dT%H:%M:%S'
default_datetime_format = '%Y-%m-%d %H:%M:%S'

binance_data_columns = ['time', 'open', 'high', 'low', 'close', 'volume']
coinbase_data_columns = ['time', 'low', 'high', 'open', 'close', 'volume']


def table(values):
    first = values[0]
    keys = list(first.keys()) if isinstance(first, dict) else range(0, len(first))
    widths = [max([len(str(v[k])) for v in values]) for k in keys]
    string = ' | '.join(['{:<' + str(w) + '}' for w in widths])
    return "\n".join([string.format(*[str(v[k]) for k in keys]) for v in values])

def listdata_to_df(data, cols):
    df = pd.DataFrame(data, columns = cols)
    # df['time'] = [binance_ts_to_datetime(time) for time in df['time']]
    # df.set_index('Time', inplace=True)
    return(df)

def coinbase_data_to_df(data):
    return(listdata_to_df(data, cols=coinbase_data_columns))

def binance_data_to_df(data):
    return(listdata_to_df(data, cols=binance_data_columns))

def ts_to_dt(ts, tzinfo='korea'):     
    # ts_binance_size = 13

    # KST = datetime.timezone(datetime.timedelta(hours=9))
    if tzinfo == 'binance':
        return(datetime.utcfromtimestamp(float(ts)/1000)) 
    elif tzinfo == 'coinbase':
        return(datetime.utcfromtimestamp(float(ts)))
    elif tzinfo == 'korea':
        return(datetime.fromtimestamp(float(ts)))

def dt_to_str(dt, format):
    return(dt.strftime(format))

def binance_ts_to_dt(time):
    return(datetime.utcfromtimestamp(float(time)/1000))

# def binance_data_to_df(data):
#     df = pd.DataFrame(data, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
#     df['Time'] = [binance_ts_to_dt(time) for time in df['Time']]
#     df.set_index('Time', inplace=True)
#     return(df)

def binance_ts_to_coinbase_datestr(ts, format=coinbase_datetime_format):
    return(dt_to_str(binance_ts_to_dt(ts), format))



# def date_from_timestamp(timestamp):
#     return(timestamp/1000)


config = configparser.ConfigParser()
config.read('../config.ini')

BINANCE_API_KEY = config['BINANCE']['API_KEY']
BINANCE_SECRET_KEY = config['BINANCE']['SECRET_KEY']
BINANCE_API_URL = config['BINANCE']['API_URL']

COINBASE_API_KEY = config['COINBASE']['API_KEY']
COINBASE_SECRET_KEY = config['COINBASE']['SECRET_KEY']
COINBASE_API_URL = config['COINBASE']['API_URL']

print('CCXT Version: ', ccxt.__version__)
exchange_id = 'binance'
binance = ccxt.binance({
    'options': {
        'adjustForTimeDifference': True,
        'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': BINANCE_API_KEY,
    'secret': BINANCE_SECRET_KEY,
})

coinbase = ccxt.coinbase({
    'options': {
        'adjustForTimeDifference': True,
        'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': COINBASE_API_KEY,
    'secret': COINBASE_SECRET_KEY,
})




CCXT Version:  1.45.42


In [2]:
# pprint(binance.api)
# pprint(binance.has)
# pprint(binance.timeframes)

# pprint(coinbase.api)
# pprint(coinbase.has)
# pprint(coinbase.timeframes)

# if binance.has['fetchOHLCV']:
#     time.sleep (binance.rateLimit / 1000) # time.sleep wants seconds
#     one_day = binance.fetch_ohlcv(symbol, '1d')


# binance returns 500 data per query 
# len(one_day)


In [7]:
def ohlcv_data(symbol, exchange, timeframe, since=None):
    msec = 1000
    minute = 60 * msec
    hour = 60 *minute
    day = 24 * hour

    if timeframe == '1m':
        tick = minute
    elif timeframe == '1h':
        tick = hour
    elif timeframe == '1d':
        tick = day 

    data = []
    now_timestamp = exchange.milliseconds()
    since_timestamp = exchange.parse8601(since)

    print('Difference: ', (now_timestamp - since_timestamp)/minute, '(',now_timestamp, '-', since_timestamp, ')')

    if (since_timestamp == None or since_timestamp > now_timestamp):
        print('Fetching candles starting from: ', since)
        data = exchange.fetch_ohlcv(symbol, timeframe)
        print('Fetching finished')  
        return(data)


    while since_timestamp < now_timestamp: 
        print('Fetching candles starting from: ', binance_ts_to_dt(since_timestamp))
        candles = exchange.fetch_ohlcv(symbol, timeframe, since=since_timestamp)
        since_timestamp = candles[-1][0] + tick 
        data += candles

    print('Fetching finished')
    return(data)

# def binance_timestamp_to_datetime(time):
#     return(datetime.utcfromtimestamp(float(time)/1000))


# def binance_data_to_dataframe(data):
#     df = pd.DataFrame(data, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
#     df['Time'] = [binance_timestamp_to_datetime(time) for time in df['Time']]
#     df.set_index('Time', inplace=True)
#     return(df)


# def ts_to_dt(ts, tzinfo='korea'):     
#     # ts_binance_size = 13

#     # KST = datetime.timezone(datetime.timedelta(hours=9))
#     if tzinfo == 'binance':
#         return(datetime.utcfromtimestamp(float(ts)/1000)) 
#     elif tzinfo == 'coinbase':
#         return(datetime.utcfromtimestamp(float(ts)))
#     elif tzinfo == 'korea':
#         return(datetime.fromtimestamp(float(ts)))

# def dt_to_str(dt, format):
#     return(dt.strftime(format))

# def binance_ts_to_coinbase_datestr(ts, format=coinbase_datetime_format):
#     return(dt_to_str(binance_timestamp_to_datetime(ts), format))

def coinbase_data_fetch_by_binance_ts(start_ts, stop_ts, granularity=60):


    # print('Fetching candles between: ', start_ts, '...', stop_ts)

    PATH = 'products/BTC-USD/candles'
    data = []

    # start_ts, stop_ts : binance timestamp
    start = binance_ts_to_coinbase_datestr(start_ts)
    stop = binance_ts_to_coinbase_datestr(stop_ts)


    while (stop_ts >= start_ts): 
    
        start = binance_ts_to_coinbase_datestr(start_ts)
        stop = binance_ts_to_coinbase_datestr(stop_ts)

        print('Fetching candles between: ', start, '...', stop)


        params = {
            'start': start,
            'stop': stop,
            'granularity': granularity
        }

        print('Fetching candles ending: ', stop)

        response = requests.get(urljoin(COINBASE_API_URL, PATH), params=params).json()
        data += response
        stop_ts = response[-1][0] * 1000
        print('last item:',  response[-1][0], '  length:', len(response))
        print('..... updated stop : ', binance_ts_to_coinbase_datestr(stop_ts))


    # print(len(str(response[0][0])))
    return(data)



# start: datetime str, stop: datetime_str 
def coinbase_data_fetch(start, stop, granularity=60):

    PATH = 'products/BTC-USD/candles'
    # COINBASE_URL = urljoin(COINBASE_API_URL, PATH)

    data = []
    #PATH = 'btc/exchange-flows/reserve'
    # URL = urljoin(API_URL, PATH)
    params = {
        'start': start,
        'stop': stop,
        'granularity': granularity
    }

    response = requests.get(urljoin(COINBASE_API_URL, PATH), params=params).json()
    # print(len(str(response[0][0])))
    return(response)

    

In [4]:
PATH = 'products/BTC-USD/candles'

In [5]:
dt_now = datetime.now()
print('dt_now:', dt_now)


ts_now = time.time()
ts_binance = binance.milliseconds()
print('ts_now:', ts_now)
print('ts_binance:', ts_binance)

dt_binance = ts_to_dt(ts_binance, 'binance')
dt_coinbase = ts_to_dt(ts_binance/1000, 'coinbase')
dt_korea = ts_to_dt(ts_now)
print('dt_korea:', dt_korea)
print('dt_binance:', dt_binance)
print('dt_coinbase:', dt_coinbase)

time_difference = dt_korea - dt_binance
print('difference:', time_difference)
# print(time_difference > 0)



dt_now: 2021-04-26 14:14:14.218047
ts_now: 1619414054.2184856
ts_binance: 1619414054218
dt_korea: 2021-04-26 14:14:14.218486
dt_binance: 2021-04-26 05:14:14.218000
dt_coinbase: 2021-04-26 05:14:14.218000
difference: 9:00:00.000486


In [8]:
test_data = ohlcv_data(binance_symbol, binance, '1m', since='2021-04-23 00:00:00')

Difference:  4634.422 ( 1619414065320 - 1619136000000 )
Fetching candles starting from:  2021-04-23 00:00:00
Fetching candles starting from:  2021-04-23 08:20:00
Fetching candles starting from:  2021-04-23 16:40:00
Fetching candles starting from:  2021-04-24 01:00:00
Fetching candles starting from:  2021-04-24 09:20:00
Fetching candles starting from:  2021-04-24 17:40:00
Fetching candles starting from:  2021-04-25 02:00:00
Fetching candles starting from:  2021-04-25 10:20:00
Fetching candles starting from:  2021-04-25 18:40:00
Fetching candles starting from:  2021-04-26 03:00:00
Fetching finished


In [9]:
print(len((test_data)))
print('difference:', (test_data[-1][0] - test_data[0][0]), '(', test_data[-1][0], '-',  test_data[0][0],  (test_data[-1][0] - test_data[0][0])/60000 +1)

4637
difference: 278160000 ( 1619414160000 - 1619136000000 4637.0


In [10]:
# test_data

In [11]:
start = binance_ts_to_coinbase_datestr(test_data[0][0])
stop = binance_ts_to_coinbase_datestr(test_data[-1][0])

print(start, stop)

2021-04-23T00:00:00 2021-04-26T05:16:00


In [12]:
start_ts = test_data[0][0]
stop_ts = test_data[-1][0]
start = binance_ts_to_coinbase_datestr(start_ts)
stop = binance_ts_to_coinbase_datestr(stop_ts)

print(start, stop)

2021-04-23T00:00:00 2021-04-26T05:16:00


In [38]:
/



# print(binance_ts_to_coinbase_datestr(stop_ts))


#  print(response)


stop_ts: 1619397300000
start_ts:  1619136000000
Fetching candles between:  2021-04-23T00:00:00 ... 2021-04-26T00:35:00
2021-04-26T05:42:00
2021-04-26T00:43:00
second call
stop_ts: 1619397720000
start_ts:  1619136000000
... updated stop:  2021-04-26T00:42:00


In [10]:
len(test_data)

483

In [11]:

# COINBASE_URL = 'https://api.pro.coinbase.com/products/BTC-USD/candles'

# #PATH = 'btc/exchange-flows/reserve'
# # URL = urljoin(API_URL, PATH)
# params = {
#     'start': start,
#     'stop': stop,
#     'granularity': 60
# }

# response = requests.get(COINBASE_URL, params=params).json()
# print(len(response))

In [14]:

# coinbase_data = coinbase_data_fetch(start, stop)
coinbase_data_1 = coinbase_data_fetch_by_binance_ts(start_ts, stop_ts)
print('length: ', len(coinbase_data_1))
print(coinbase_data_1)


# since='2021-04-18 00:00:00'

# # stop = datetime.strftime(since, format)
# # stop


# # COINBASE_URL = 'https://api.pro.coinbase.com/products/BTC-USD/candles'

# #PATH = 'btc/exchange-flows/reserve'
# # URL = urljoin(API_URL, PATH)
# params = {
#     'start': start,
#     'stop': stop,
#     'granularity': 60
# }

# response = requests.get(COINBASE_API_URL, params=params).json()
# # print(response)
# # print(response['status'])
# # print(response['result'])
# # print(len(response))
# print(len(str(response[0][0])))

Fetching candles between:  2021-04-23T00:00:00 ... 2021-04-23T08:02:00
Fetching candles ending:  2021-04-23T08:02:00
last item: 1619147160   length: 300
..... updated stop :  2021-04-23T03:06:00
Fetching candles between:  2021-04-23T00:00:00 ... 2021-04-23T03:06:00
Fetching candles ending:  2021-04-23T03:06:00
last item: 1619147160   length: 300
..... updated stop :  2021-04-23T03:06:00
Fetching candles between:  2021-04-23T00:00:00 ... 2021-04-23T03:06:00
Fetching candles ending:  2021-04-23T03:06:00
last item: 1619147160   length: 300
..... updated stop :  2021-04-23T03:06:00
Fetching candles between:  2021-04-23T00:00:00 ... 2021-04-23T03:06:00
Fetching candles ending:  2021-04-23T03:06:00
last item: 1619147160   length: 300
..... updated stop :  2021-04-23T03:06:00
Fetching candles between:  2021-04-23T00:00:00 ... 2021-04-23T03:06:00
Fetching candles ending:  2021-04-23T03:06:00
last item: 1619147160   length: 300
..... updated stop :  2021-04-23T03:06:00
Fetching candles between:

KeyboardInterrupt: 

In [16]:
print(ts_to_dt(coinbase_data[0][0]))
print(ts_to_dt(coinbase_data[-1][0]))

2021-04-23 16:04:00
2021-04-23 11:05:00


In [ ]:
df = binance_data_to_df(test_data)

print(len(df))

In [ ]:
df
# print(df.iloc[-1, :])
# print(df.iloc[-1, 0])
# print(df.iloc[-1, 1])
# print(df.iloc[-1, 2])
# print(df.iloc[-1, 3])
# print(df.iloc[-1, 4])

# print(df[-1:].index)

In [ ]:
ts_to_dt(test_data[-1][0]/1000, 'coinbase')

In [ ]:
coinbase_symbol = 'BTC/USD'
coinbase.load_markets()

In [ ]:
dt_start = binance_timestamp_to_datetime(test_data[0][0])
dt_end = binance_timestamp_to_datetime(test_data[-1][0])

In [ ]:
print(dt_start)
print(dt_end)

In [ ]:
# def coinbase_ohlcv_data(symbol, timeframe, start, stop):
type(df[-1:].index)


In [ ]:

# 2018-07-10T12:00:00 
coinbase_time_format = '%Y-%m-%dT%H:%M:%S'
default_time_format = '%Y-%m-%d %H:%M:%S'


print(dt_to_str(dt_start, coinbase_time_format))

In [ ]:





since='2021-04-18 00:00:00'

# stop = datetime.strftime(since, format)
# stop


COINBASE_URL = 'https://api.pro.coinbase.com/products/BTC-USD/candles'

#PATH = 'btc/exchange-flows/reserve'
# URL = urljoin(API_URL, PATH)
params = {
    'start': '2018-07-10T12:00:00',
    'stop': '2018-07-15T12:00:00',
    'granularity': 60
}

response = requests.get(COINBASE_URL, params=params).json()
print(response)
# print(response['status'])
# print(response['result'])
print(len(response))

In [ ]:
COINBASE_URL = 'https://api.pro.coinbase.com/time'

#PATH = 'btc/exchange-flows/reserve'
# URL = urljoin(API_URL, PATH)
# params = {
#     'start': '2018-07-10T12:00:00',
#     'stop': '2018-07-15T12:00:00',
#     'granularity': 60
# }

response = requests.get(COINBASE_URL).json()
print(response)

